In [ ]:
import os
import random
import glob 
import cv2
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import timm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from facenet_pytorch import MTCNN, InceptionResnetV1

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # CuDNN 결정론적 옵션 설정
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 원하는 시드 값 설정
seed = 42
set_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 디바이스 설정
print(device)

In [ ]:
people_list = os.listdir('./real_test')
print(len(people_list))
print(people_list[:5])

In [ ]:
left_image_paths = glob.glob(os.path.join('crop_real_test','pair*','L.jpg'))
right_image_paths = glob.glob(os.path.join('crop_real_test','pair*','R.jpg'))

In [ ]:
print(len(left_image_paths))
print(left_image_paths[:10])
print(len(right_image_paths))
print(right_image_paths[:10])

In [ ]:
class TestDataset(Dataset):
    def __init__(self,transform):
        self.transform = transform
        
    def __len__(self):
        return 75200

    def __getitem__(self,idx):
        left_img_path = os.path.join('crop_real_test',f'pair{idx}','L.jpg')
        right_img_path = os.path.join('crop_real_test',f'pair{idx}','R.jpg')
        
        left_img = cv2.imread(left_img_path)
        left_img = cv2.cvtColor(left_img,cv2.COLOR_BGR2RGB)

        right_img = cv2.imread(right_img_path)
        right_img = cv2.cvtColor(right_img,cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            left_img = self.transform(image=left_img)['image']
            right_img = self.transform(image=right_img)['image']
            
        return left_img, right_img

In [ ]:
transform = A.Compose([
    A.Resize(112,112, p=1.0),
    A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225], max_pixel_value=255, p=1.0),
    ToTensorV2(),
])

In [ ]:
test_dataset = TestDataset(transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=512,num_workers=8, shuffle=False)

In [ ]:
print(len(test_dataset),len(test_dataloader))

In [ ]:
model = InceptionResnetV1(classify=False, pretrained='vggface2').to(device)
# model_file = torch.load("./pth_file/InceptionResnetV1_sphere_best_model.pt")
# model_file = torch.load("./pth_file/InceptionResnetV1_arc_best_model.pt")
model_file = torch.load("./pth_file/InceptionResnetV1_cos_best_model.pt")
model.load_state_dict(model_file)

In [ ]:
model.eval()

answer_list = []

with torch.no_grad():
    for images,second_images in tqdm(test_dataloader):
        images = images.to(device)
        second_images = second_images.to(device)
                
        outputs = model(images) # (Batch,512)
        second_outputs = model(second_images)
        cos_sim = F.cosine_similarity(outputs, second_outputs) # (Batch,)
        answer_list.append(cos_sim)
    answer_list = torch.cat(answer_list,dim=0)
print(answer_list)

In [ ]:
submission = pd.read_csv("./submission/main_sample_submission.csv")

print(submission)

In [ ]:
# 원본 데이터프레임을 복사
submission_copy = submission.copy()

# 'path' 열에서 숫자를 추출하고 새로운 열 'pair_number'로 추가
submission_copy['pair_number'] = submission_copy['path'].apply(lambda x: int(x.split('/')[0][4:]))

# 'pair_number' 열을 기준으로 데이터프레임 정렬
submission_copy = submission_copy.sort_values(by='pair_number')

# 정렬된 데이터프레임에서 필요한 열만 선택
submission = submission_copy[['path', 'answer']]
print(submission)

In [ ]:
submission = submission.reset_index(drop=True)
print(submission)

In [ ]:
submission['answer'] = answer_list.tolist()

submission.to_csv('./submission/main_efficient_bo_cos_submission.csv', index=False)